In [121]:
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix
import scipy.spatial.distance

In [122]:
NFL2022 = pd.read_csv('Full_NFL_Stats_2022.csv')
NFL2022.head()

,Unnamed: 0,teamName,displayName,description,abbreviation,value,perGameValue,rank
0,0,Falcons,Fumbles,The number of times a player/team has fumbled ...,FUM,21.0,1.0,13.0
1,1,Falcons,Fumbles Lost,The number of times a fumble is recovered by t...,LST,12.0,NaN,4.0
2,2,Falcons,Forced Fumbles,The total number of forced fumbles.,FF,8.0,NaN,62.0
3,3,Falcons,Fumbles Recovered,The number of fumbles recovered.,FR,7.0,NaN,23.0
4,4,Falcons,Fumbles Touchdowns,The number of times a fumbles is recovered and...,FTD,1.0,NaN,8.0


In [123]:
NFL2022_df = NFL2022.drop(columns=['Unnamed: 0', 'perGameValue'])

In [124]:
NFL2022_df.head()

,teamName,displayName,description,abbreviation,value,rank
0,Falcons,Fumbles,The number of times a player/team has fumbled ...,FUM,21.0,13.0
1,Falcons,Fumbles Lost,The number of times a fumble is recovered by t...,LST,12.0,4.0
2,Falcons,Forced Fumbles,The total number of forced fumbles.,FF,8.0,62.0
3,Falcons,Fumbles Recovered,The number of fumbles recovered.,FR,7.0,23.0
4,Falcons,Fumbles Touchdowns,The number of times a fumbles is recovered and...,FTD,1.0,8.0


In [125]:
NFL2022_df.sort_values(by=['rank'], ascending=False)

,teamName,displayName,description,abbreviation,value,rank
4983,Giants,Quarterback Rating,Quarterback Rating.,RAT,91.111664,114.0
4964,Giants,Passer Rating,Passer Rating,RTG,91.111664,114.0
7175,Buccaneers,Quarterback Rating,Quarterback Rating.,RAT,89.777500,106.0
7156,Buccaneers,Passer Rating,Passer Rating,RTG,89.777500,106.0
6353,Chargers,Quarterback Rating,Quarterback Rating.,RAT,93.334990,102.0
...,...,...,...,...,...,...
6352,Chargers,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,6.112000,NaN
7174,Buccaneers,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,5.932730,NaN
7448,Commanders,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,5.767442,NaN
7722,Panthers,Net Yards Per Pass Attempt,The average number net yards (taking sack yard...,NYDS/PA,6.077079,NaN


In [126]:
NFL2022_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8768 entries, 0 to 8767
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   teamName      8768 non-null   object 
 1   displayName   8768 non-null   object 
 2   description   8768 non-null   object 
 3   abbreviation  8768 non-null   object 
 4   value         8768 non-null   float64
 5   rank          8748 non-null   float64
dtypes: float64(2), object(4)
memory usage: 411.1+ KB


In [127]:
NFL2021 = pd.read_csv('Full_NFL_Stats_2021.csv')
NFL2021.head()

,Unnamed: 0,teamName,displayName,description,abbreviation,value,rank
0,0,Falcons,Fumbles,The number of times a player/team has fumbled ...,FUM,24.0,8.0
1,1,Falcons,Fumbles Lost,The number of times a fumble is recovered by t...,LST,11.0,4.0
2,2,Falcons,Forced Fumbles,The total number of forced fumbles.,FF,18.0,14.0
3,3,Falcons,Fumbles Recovered,The number of fumbles recovered.,FR,8.0,11.0
4,4,Falcons,Fumbles Touchdowns,The number of times a fumbles is recovered and...,FTD,0.0,12.0


In [128]:
NFL2021_df = NFL2021.drop(columns='Unnamed: 0')

In [129]:
NFL2021_df.sort_values(by=['rank'], ascending=False)

,teamName,displayName,description,abbreviation,value,rank
2791,Colts,Quarterback Rating,Quarterback Rating.,RAT,93.475840,102.0
2772,Colts,Passer Rating,Passer Rating,RTG,93.475840,102.0
4142,Vikings,Passer Rating,Passer Rating,RTG,101.629166,98.0
4161,Vikings,Quarterback Rating,Quarterback Rating.,RAT,101.629166,98.0
6079,Steelers,Quarterback Rating,Quarterback Rating.,RAT,85.272510,93.0
...,...,...,...,...,...,...
8753,Texans,Fumbles Lost,The number of times the ball has been fumbled ...,LST,7.000000,NaN
8762,Texans,Total Giveaways,The total number of giveaways.,TGV,22.000000,NaN
8765,Texans,Total Takeaways,The total number of takeaways.,TT,25.000000,NaN
8766,Texans,Total Drives,The total number of drives.,D,0.000000,NaN


In [130]:
NFL2022dr_df = NFL2022_df.drop(columns=['displayName', 'description', 'abbreviation'])

In [131]:
NFL2022dr_df = NFL2022dr_df.dropna()

In [132]:
NFL2022in_df = NFL2022dr_df.set_index('teamName')
NFL2022in_df.head()

,value,rank
teamName,,
Falcons,21.0,13.0
Falcons,12.0,4.0
Falcons,8.0,62.0
Falcons,7.0,23.0
Falcons,1.0,8.0


In [133]:
tackles = pd.read_csv('TotalTackles.csv')
tackles.drop(columns='Unnamed: 0', inplace=True)
tackles.head()

,PlayerName,Position,assistTackles,avgInterceptionYards,avgSackYards,avgStuffYards,blockedFieldGoalTouchdowns,blockedPuntTouchdowns,hurries,kicksBlocked,...,safeties,soloTackles,stuffs,stuffYards,tacklesForLoss,tacklesYardsLost,teamGamesPlayed,totalTackles,yardsAllowed,pointsAllowed
0,Foyesade Oluokun,LB,0.000,6.000,1.700,0.0,0.0,0.0,0.0,0.0,...,128.0,18.5,17.0,12.0,26.0,17.0,184.0,0.0,0.0,0.0
1,Nick Bolton,LB,7.500,9.500,1.500,0.0,0.0,0.0,0.0,1.0,...,108.0,12.0,12.0,9.0,22.0,17.0,180.0,0.0,0.0,0.0
2,Roquan Smith,LB,13.667,5.000,1.857,0.0,0.0,0.0,0.0,1.0,...,103.0,18.5,13.0,11.0,29.0,17.0,169.0,0.0,0.0,0.0
3,Zaire Franklin,LB,0.000,5.833,1.900,0.0,0.0,0.0,0.0,0.0,...,102.0,14.5,19.0,12.0,24.0,17.0,167.0,0.0,0.0,0.0
4,Alex Singleton,LB,0.000,0.000,2.000,0.0,0.0,0.0,0.0,0.0,...,100.0,7.5,12.0,6.0,10.0,17.0,163.0,0.0,0.0,0.0


In [134]:
sacks = pd.read_csv('SacksData.csv')
sacks.drop(columns='Unnamed: 0', inplace=True)
sacks.head()

,PlayerName,Position,assistTackles,avgInterceptionYards,avgSackYards,avgStuffYards,blockedFieldGoalTouchdowns,blockedPuntTouchdowns,hurries,kicksBlocked,...,safeties,soloTackles,stuffs,stuffYards,tacklesForLoss,tacklesYardsLost,teamGamesPlayed,totalTackles,yardsAllowed,pointsAllowed
0,Nick Bosa,DE,0.0,7.432,2.667,0.0,0.0,0.0,0.0,0.0,...,41.0,5.0,8.0,19.0,129.0,17.0,51.0,0.0,0.0,0.0
1,Haason Reddick,LB,0.0,8.281,1.667,0.0,0.0,0.0,0.0,0.0,...,35.0,4.0,5.0,11.0,64.0,17.0,49.0,0.0,0.0,0.0
2,Myles Garrett,DE,0.0,5.688,3.000,0.0,0.0,0.0,0.0,0.0,...,37.0,8.5,12.0,18.0,85.0,17.0,60.0,0.0,0.0,0.0
3,Chris Jones,DT,0.0,6.806,2.400,0.0,0.0,0.0,0.0,0.0,...,30.0,5.5,12.0,17.0,94.0,17.0,44.0,0.0,0.0,0.0
4,Matthew Judon,LB,0.0,6.613,1.333,0.0,0.0,0.0,0.0,0.0,...,36.0,7.0,4.0,14.0,99.0,17.0,60.0,0.0,0.0,0.0


### LB Similarity

In [135]:
lbstt = tackles.loc[tackles['Position'] == 'LB']
lbstt = lbstt.set_index('PlayerName')
lbstt = lbstt.reset_index()
lbstt.head()

,PlayerName,Position,assistTackles,avgInterceptionYards,avgSackYards,avgStuffYards,blockedFieldGoalTouchdowns,blockedPuntTouchdowns,hurries,kicksBlocked,...,safeties,soloTackles,stuffs,stuffYards,tacklesForLoss,tacklesYardsLost,teamGamesPlayed,totalTackles,yardsAllowed,pointsAllowed
0,Foyesade Oluokun,LB,0.000,6.000,1.700,0.0,0.0,0.0,0.0,0.0,...,128.0,18.5,17.0,12.0,26.0,17.0,184.0,0.0,0.0,0.0
1,Nick Bolton,LB,7.500,9.500,1.500,0.0,0.0,0.0,0.0,1.0,...,108.0,12.0,12.0,9.0,22.0,17.0,180.0,0.0,0.0,0.0
2,Roquan Smith,LB,13.667,5.000,1.857,0.0,0.0,0.0,0.0,1.0,...,103.0,18.5,13.0,11.0,29.0,17.0,169.0,0.0,0.0,0.0
3,Zaire Franklin,LB,0.000,5.833,1.900,0.0,0.0,0.0,0.0,0.0,...,102.0,14.5,19.0,12.0,24.0,17.0,167.0,0.0,0.0,0.0
4,Alex Singleton,LB,0.000,0.000,2.000,0.0,0.0,0.0,0.0,0.0,...,100.0,7.5,12.0,6.0,10.0,17.0,163.0,0.0,0.0,0.0


In [136]:
lbstt = lbstt.drop(columns='Position')
lbttsimdex_df = lbstt.set_index(['PlayerName'])

In [137]:
t_player = 'Foyesade Oluokun'
t_player_stat = lbttsimdex_df.loc[t_player]
dist = scipy.spatial.distance.cdist(lbttsimdex_df, [t_player_stat], metric = "cosine")[:,0]
query_distances = list(zip(lbstt.index, dist))

In [138]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:16]:
    print(lbstt['PlayerName'].iloc[sim_play], sim_stats)

Foyesade Oluokun 0.0
Zaire Franklin 0.0027508244068337184
Eric Kendricks 0.0028219192369823842
Bobby Okereke 0.003987542895135809
Nick Bolton 0.004287539420890885
Tremaine Edmunds 0.004495210550057971
T.J. Edwards 0.004573338283919592
Alex Anzalone 0.004702219708321853
Jordyn Brooks 0.004782288089118514
Cody Barton 0.005725631477929705
Malcolm Rodriguez 0.006265427202595375
Roquan Smith 0.00657164961235257
Quay Walker 0.006829748653799594
Alex Singleton 0.006929882841380186
Dre Greenlaw 0.007526802476184313
C.J. Mosley 0.008105301934881237


In [139]:
lbssc = sacks.loc[sacks['Position'] == 'LB']
lbssc = lbssc.set_index('PlayerName')
lbssc = lbssc.reset_index()
lbssc.head()

,PlayerName,Position,assistTackles,avgInterceptionYards,avgSackYards,avgStuffYards,blockedFieldGoalTouchdowns,blockedPuntTouchdowns,hurries,kicksBlocked,...,safeties,soloTackles,stuffs,stuffYards,tacklesForLoss,tacklesYardsLost,teamGamesPlayed,totalTackles,yardsAllowed,pointsAllowed
0,Haason Reddick,LB,0.0,8.281,1.667,0.0,0.0,0.0,0.0,0.0,...,35.0,4.0,5.0,11.0,64.0,17.0,49.0,0.0,0.0,0.0
1,Matthew Judon,LB,0.0,6.613,1.333,0.0,0.0,0.0,0.0,0.0,...,36.0,7.0,4.0,14.0,99.0,17.0,60.0,0.0,0.0,0.0
2,Alex Highsmith,LB,0.0,8.138,5.250,0.0,0.0,0.0,0.0,0.0,...,38.0,6.5,21.0,12.0,91.0,17.0,63.0,0.0,0.0,0.0
3,Micah Parsons,LB,0.0,6.556,1.750,0.0,0.0,0.0,0.0,0.0,...,42.0,4.0,7.0,13.0,66.0,17.0,65.0,0.0,0.0,0.0
4,Josh Uche,LB,0.0,4.304,0.000,0.0,0.0,0.0,0.0,0.0,...,23.0,0.0,0.0,9.0,47.0,17.0,27.0,0.0,0.0,0.0


In [140]:
lbssc = lbssc.drop(columns='Position')
lbscsimdex_df = lbssc.set_index(['PlayerName'])

In [145]:
t_player = 'Haason Reddick'
t_player_stat = lbscsimdex_df.loc[t_player]
dist = scipy.spatial.distance.cdist(lbscsimdex_df, [t_player_stat], metric = "cosine")[:,0]
query_distances = list(zip(lbssc.index, dist))

In [146]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:16]:
    print(lbssc['PlayerName'].iloc[sim_play], sim_stats)

Haason Reddick 1.1102230246251565e-16
Darrell Taylor 0.008981820581654754
Justin Houston 0.017630605964722212
Alex Highsmith 0.025156148188099303
Bradley Chubb 0.025412037578418367
Melvin Ingram 0.0260457070774176
Micah Parsons 0.03735301097551991
Matthew Judon 0.042428924095662124
Von Miller 0.04617582317401292
Josh Uche 0.04902161365874225
Azeez Ojulari 0.05512409923721695
Rashan Gary 0.05761264852219017
Khalil Mack 0.06325189919262242
Preston Smith 0.06774348653438245
James Houston 0.06851533701330437
Samson Ebukam 0.06999042608903638


### DB Similarity

In [167]:
dbs = tackles.loc[tackles['Position'] == 'CB']
dbs = dbs.set_index('PlayerName')
dbs = dbs.reset_index()
dbs.head()

,PlayerName,Position,assistTackles,avgInterceptionYards,avgSackYards,avgStuffYards,blockedFieldGoalTouchdowns,blockedPuntTouchdowns,hurries,kicksBlocked,...,safeties,soloTackles,stuffs,stuffYards,tacklesForLoss,tacklesYardsLost,teamGamesPlayed,totalTackles,yardsAllowed,pointsAllowed
0,L'Jarius Sneed,CB,19.00,8.857,2.000,0.0,0.0,0.0,0.0,1.0,...,75.0,4.5,8.0,5.0,11.0,17.0,108.0,0.0,0.0,0.0
1,Taron Johnson,CB,2.00,0.000,1.667,0.0,0.0,0.0,0.0,1.0,...,67.0,6.0,10.0,6.0,4.0,16.0,90.0,0.0,0.0,0.0
2,Desmond King II,CB,0.00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
3,Jalen Ramsey,CB,13.25,8.500,1.333,0.0,0.0,0.0,0.0,1.0,...,64.0,4.5,4.0,4.0,11.0,17.0,88.0,0.0,0.0,0.0
4,Charvarius Ward,CB,9.00,0.000,1.667,0.0,0.0,0.0,0.0,1.0,...,59.0,4.5,5.0,3.0,1.0,17.0,87.0,0.0,0.0,0.0


In [168]:
dbs = dbs.drop(columns='Position')
dbsimdex_df = dbs.set_index(['PlayerName'])

In [169]:
t_player = "L'Jarius Sneed"
t_player_stat = dbsimdex_df.loc[t_player]
dist = scipy.spatial.distance.cdist(dbsimdex_df, [t_player_stat], metric = "cosine")[:,0]
query_distances = list(zip(dbs.index, dist))

In [170]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:16]:
    print(dbs['PlayerName'].iloc[sim_play], sim_stats)

L'Jarius Sneed 0.0
Jalen Ramsey 0.007268891014888479
Arthur Maulet 0.01138473835734366
Marlon Humphrey 0.014502468039428007
Deommodore Lenoir 0.021436782957368394
Coby Bryant 0.024322503000657525
Marcus Peters 0.026157460350360484
Eric Rowe 0.027320289805224385
Nate Hobbs 0.027910617235572865
Martin Emerson Jr. 0.03238589416274085
Charvarius Ward 0.033215540271458344
K'Waun Williams 0.03525250270346214
Jeff Okudah 0.03622340200347196
Benjamin St-Juste 0.036969197423821876
Troy Hill 0.03722338871765529
Kenny Moore II 0.03775936757806009


### DLine Similarity

In [154]:
pos = 'DE', 'DT'
dlinett = tackles.loc[tackles['Position'].isin(pos)]
dlinett = dlinett.set_index('PlayerName')
dlinett = dlinett.reset_index()
dlinett.head()

,PlayerName,Position,assistTackles,avgInterceptionYards,avgSackYards,avgStuffYards,blockedFieldGoalTouchdowns,blockedPuntTouchdowns,hurries,kicksBlocked,...,safeties,soloTackles,stuffs,stuffYards,tacklesForLoss,tacklesYardsLost,teamGamesPlayed,totalTackles,yardsAllowed,pointsAllowed
0,Christian Wilkins,DT,0.0,7.000,2.071,0.0,0.0,0.0,0.0,0.0,...,59.0,17.5,29.0,16.0,40.0,17.0,98.0,0.0,0.0,0.0
1,Maxx Crosby,DE,0.0,8.160,2.250,0.0,0.0,0.0,1.0,0.0,...,58.0,20.0,27.0,22.0,104.0,17.0,89.0,0.0,0.0,0.0
2,Cameron Heyward,DT,0.0,6.905,2.167,0.0,0.0,0.0,0.0,0.0,...,39.0,11.5,13.0,14.0,58.0,17.0,74.0,0.0,0.0,0.0
3,DeForest Buckner,DT,0.0,6.063,2.000,0.0,0.0,0.0,0.0,0.0,...,44.0,12.0,12.0,11.0,43.0,17.0,74.0,0.0,0.0,0.0
4,Zach Sieler,DT,0.0,9.000,2.667,0.0,0.0,0.0,0.0,0.0,...,41.0,10.0,8.0,7.0,41.0,17.0,70.0,0.0,0.0,0.0


In [171]:
dlinett = dlinett.drop(columns='Position')
dlinettimdex_df = dlinett.set_index(['PlayerName'])

In [172]:
t_player = "Christian Wilkins"
t_player_stat = dlinettimdex_df.loc[t_player]
dist = scipy.spatial.distance.cdist(dlinettimdex_df, [t_player_stat], metric = "cosine")[:,0]
query_distances = list(zip(dlinett.index, dist))

In [173]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:16]:
    print(dlinett['PlayerName'].iloc[sim_play], sim_stats)

Christian Wilkins 0.0
Grover Stewart 0.005884971178452769
Sebastian Joseph-Day 0.01597064438103013
Larry Ogunjobi 0.02575481627154974
Justin Jones 0.026536259183034705
Shelby Harris 0.0276314770470244
Al Woods 0.02917655334190017
Roy Lopez 0.031099209826474405
Zach Sieler 0.03135957977279835
Maliek Collins 0.03550912365887282
DeMarcus Lawrence 0.035760475684381454
Rasheem Green 0.03596420867137262
Jordan Elliott 0.036208558063924645
Justin Madubuike 0.0387786315389641
Roy Robertson-Harris 0.042261008731865335
DaVon Hamilton 0.044374596415347356


In [174]:
pos = 'DE', 'DT'
dlinesc = sacks.loc[sacks['Position'].isin(pos)]
dlinesc = dlinesc.set_index('PlayerName')
dlinesc = dlinesc.reset_index()
dlinesc.head()

,PlayerName,Position,assistTackles,avgInterceptionYards,avgSackYards,avgStuffYards,blockedFieldGoalTouchdowns,blockedPuntTouchdowns,hurries,kicksBlocked,...,safeties,soloTackles,stuffs,stuffYards,tacklesForLoss,tacklesYardsLost,teamGamesPlayed,totalTackles,yardsAllowed,pointsAllowed
0,Nick Bosa,DE,0.0,7.432,2.667,0.0,0.0,0.0,0.0,0.0,...,41.0,5.0,8.0,19.0,129.0,17.0,51.0,0.0,0.0,0.0
1,Myles Garrett,DE,0.0,5.688,3.000,0.0,0.0,0.0,0.0,0.0,...,37.0,8.5,12.0,18.0,85.0,17.0,60.0,0.0,0.0,0.0
2,Chris Jones,DT,0.0,6.806,2.400,0.0,0.0,0.0,0.0,0.0,...,30.0,5.5,12.0,17.0,94.0,17.0,44.0,0.0,0.0,0.0
3,J.J. Watt,DE,0.0,7.520,2.000,0.0,0.0,0.0,0.0,0.0,...,30.0,11.5,14.0,18.0,96.0,17.0,39.0,0.0,0.0,0.0
4,Maxx Crosby,DE,0.0,8.160,2.250,0.0,0.0,0.0,1.0,0.0,...,58.0,20.0,27.0,22.0,104.0,17.0,89.0,0.0,0.0,0.0


In [175]:
dlinesc = dlinesc.drop(columns='Position')
dlinescimdex_df = dlinesc.set_index(['PlayerName'])

In [176]:
t_player = "Nick Bosa"
t_player_stat = dlinescimdex_df.loc[t_player]
dist = scipy.spatial.distance.cdist(dlinescimdex_df, [t_player_stat], metric = "cosine")[:,0]
query_distances = list(zip(dlinesc.index, dist))

In [177]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:16]:
    print(dlinesc['PlayerName'].iloc[sim_play], sim_stats)

Nick Bosa 0.0
Chris Jones 0.0035491949951625967
J.J. Watt 0.007774157018547778
Quinnen Williams 0.012834851789954693
Yannick Ngakoue 0.019312429058163882
Myles Garrett 0.020580569716193153
AJ Epenesa 0.020766477990339594
Denico Autry 0.02502041586430359
Quinton Jefferson 0.02789542740046258
Brandon Graham 0.029085305777819426
Vita Vea 0.029977495881471583
Dante Fowler Jr. 0.030670348035748862
DeMarcus Walker 0.030801411948735535
Montez Sweat 0.031139032631660624
Brian Burns 0.03135700228154248
Jerry Hughes 0.03152574683293807
